In [61]:
import pandas as pd
import requests
import sys
import time

#noaa_key = "JOwIFUXjlWBqtstSqsSaKYDMzShAccQe"
#usda_key = "8D5EDCA6-733D-3ACA-AF06-BA5BB3938B4F"

# Rechelle's keys (currently active)
noaa_key= "FtekNkLHUPsNabCSpEVMXldengLWOCAm"
usda_key= "6B727F30-94D3-3E2A-9186-6F7EB2F90578"

In [62]:
import requests

def get_usda_data(usda_key, year):
  # API endpoint
  url = "https://quickstats.nass.usda.gov/api/api_GET/"

  params = {
    "key": usda_key,
    "commodity_desc": "CORN",
    "statisticcat_desc": "YIELD",
    "unit_desc": "BU / ACRE",
    "state_alpha": "MI",
    "year": year,
    "short_desc": "CORN, GRAIN - YIELD, MEASURED IN BU / ACRE",
    "agg_level_desc": "COUNTY",
    "format": "JSON"
  }

  response = requests.get(url, params=params)

  if response.status_code == 200:
    data = response.json()
    return pd.DataFrame(data['data'])
  else:
    print(f"Request failed with status code {response.status_code}")
    return None

In [63]:
# def is_valid_data(data):
#   return data and isinstance(data, dict) and "results" in data and data["results"]

# # Sate in USDA data is state_ansi
# def get_raw_weather(state_ansi, county_ansi, startdate, enddate):
#   url = "https://www.ncdc.noaa.gov/cdo-web/api/v2/data"

#   headers = {
#   'token': noaa_key
#   }

#   # Convert startdate and enddate to datetime objects
#   startdate = pd.to_datetime(startdate)
#   enddate = pd.to_datetime(enddate)

#   # Calculate the number of days between startdate and enddate
#   num_days = (enddate - startdate).days

#   # Check if the number of days is greater than 30
#   if num_days > 30:
#     # Split the date range into multiple requests
#     requests_list = []
#     current_date = startdate
#     while current_date <= enddate:
#       # Calculate the end date for the current request
#       current_enddate = current_date + pd.DateOffset(days=30)
#       if current_enddate > enddate:
#         current_enddate = enddate

#       # Create the parameters for the current request
#       params = {
#         "datasetid": "GHCND",
#         "startdate": current_date.strftime("%Y-%m-%d"),
#         "enddate": current_enddate.strftime("%Y-%m-%d"),
#         "units": "standard",
#         "limit": "1000", # 1000 is the max limit
#         "locationid": f"FIPS:{state_ansi}{county_ansi}",
#         'datatypeid': ['TMAX', 'TMIN']
#       }

#       # Make the request
#       print(f"FIPS:{state_ansi}{county_ansi}")
#       response = requests.get(url, headers=headers, params=params)

#       if response.status_code == 200:
#         data = response.json()
#         if is_valid_data(data):
#           # Append the results to the list
#           requests_list.append(pd.DataFrame(data["results"]))
#         else:
#           print(f"No data found for FIPS:{params['locationid']}")
#       elif response.status_code == 503:
#         print("Server error. Retrying in 3 seconds...")
#         time.sleep(3)
#         response = requests.get(url, headers=headers, params=params)
#         if response.status_code == 200:
#           data = response.json()
#           if is_valid_data(data):
#             # Append the results to the list
#             requests_list.append(pd.DataFrame(data["results"]))
#           else:
#             print(f"No data found for FIPS:{params['locationid']} after retry")
#         else:
#           print(response.text)
#       else:
#         print(response.text)

#       # Move to the next date range
#       current_date = current_enddate + pd.DateOffset(days=1)

#     # Concatenate the results from all requests
#     if requests_list:
#       return pd.concat(requests_list)
#     else:
#       return None
#   else:
#     # Single request for the entire date range
#     params = {
#       "datasetid": "GHCND",
#       "startdate": startdate.strftime("%Y-%m-%d"),
#       "enddate": enddate.strftime("%Y-%m-%d"),
#       "units": "standard",
#       "limit": "1000",
#       "locationid": f"FIPS:{state_ansi}{county_ansi}",
#       'datatypeid': ['TMAX', 'TMIN']
#     }
#     print(f"FIPS:{state_ansi}{county_ansi}")
#     response = requests.get(url, headers=headers, params=params)

#     if response.status_code == 200:
#       data = response.json()
#       # Return data as a DataFrame
#       print(data)
#       # Check if 'results' key is present in the response
#       if is_valid_data(data):
#         return pd.DataFrame(data["results"])
#       else:
#         print(f"No data found for FIPS:{params['locationid']}")
#         return None
#     else:
#       print(response.text)
#       return None

In [64]:
# weather_raw = get_raw_weather("26", "021", "2022-12-20T00:00:00", "2022-12-31T00:00:00")

# # Save the raw weather data to a CSV file
# weather_raw.to_csv("weather_raw.csv", index=False)

In [65]:
# GDD calculation function
def calculate_gdd(df, base_temp=50, upper_temp=86):
    """
    Calculate Growing Degree Days (GDD) for corn.
    """
    df['TMAX'] = df['TMAX'].clip(lower=base_temp, upper=upper_temp)
    df['TMIN'] = df['TMIN'].clip(lower=base_temp)
    df['TAVG'] = (df['TMAX'] + df['TMIN']) / 2
    df['GDD'] = df['TAVG'] - base_temp
    return df

In [66]:
def clean_weather(df,state_ansi,county_ansi, base_temp=50, upper_temp=86):
    # For each day in date column average the TMAX and TMIN into their own columns across all stations
    df["date"] = pd.to_datetime(df["date"])
    df["TMAX"] = df["value"].where(df["datatype"] == "TMAX")
    df["TMIN"] = df["value"].where(df["datatype"] == "TMIN")
    
    df = df.groupby("date").agg({"TMAX": "mean", "TMIN": "mean"}).reset_index()

    # Calculate GDD
    df = calculate_gdd(df, base_temp, upper_temp)

    # add state and county columns
    df["state_ansi"] = state_ansi
    df["county_ansi"] = county_ansi
    
    return df

In [67]:
weather = clean_weather(weather_raw, "26", "145")

# Print the first 25 rows of the weather DataFrame
weather.head(50)
weather.to_csv("weather.csv", index=False)

In [68]:
# for each state_ansi county_ansi combo in the USDA data, get the weather data

def get_weather_data(usda_df):
    weather_data = []
    for index, row in usda_df.iterrows():
        state_ansi = row["state_ansi"]
        county_ansi = row["county_ansi"]
        #time.sleep(1)
        weather_raw = get_raw_weather(state_ansi, county_ansi, "2022-01-20T00:00:00", "2022-12-31T00:00:00")
        if weather_raw is not None:
            weather = clean_weather(weather_raw, state_ansi, county_ansi)
            weather_data.append(weather)
    return pd.concat(weather_data)

# weather_data = get_weather_data(usda_df)
# weather_data.to_csv("weather_data.csv", index=False)

In [69]:
# Count the number of unique items in column county for weather_data
# weather_data["county"].nunique()

In [70]:
# Define a list of 50 years from 1970 to 2019
# Get USDA data for each year
# Get get_weather_data for each year
# Save the data to a CSV file

years = range(2019, 2022)


# for year in years:
#   print(year)
#   usda_data = get_usda_data(usda_key, year)
#   usda_data.to_csv(f"usda_data_{year}.csv", index=False)
#   weather_data=get_weather_data(usda_data)
#   weather_data.to_csv(f"weather_data_{year}.csv", index=False)

In [71]:
import time
import pandas as pd
import requests
from requests.exceptions import SSLError

def is_valid_data(data):
    return data and isinstance(data, dict) and "results" in data and data["results"]

def get_raw_weather(state_ansi, county_ansi, startdate, enddate):
    url = "https://www.ncdc.noaa.gov/cdo-web/api/v2/data"
    headers = {
        'token': noaa_key
    }

    startdate = pd.to_datetime(startdate)
    enddate = pd.to_datetime(enddate)
    num_days = (enddate - startdate).days

    counties_to_retry = []

    def make_request(params):
        retries = 3
        backoff = 3
        for i in range(retries):
            try:
                response = requests.get(url, headers=headers, params=params)
                if response.status_code == 200:
                    data = response.json()
                    if is_valid_data(data):
                        return pd.DataFrame(data["results"])
                    else:
                        print(f"No data found for FIPS:{params['locationid']}")
                        counties_to_retry.append(params['locationid'])
                        return None
                elif response.status_code == 429:
                    print(f"Rate limit exceeded. Retrying in {backoff} seconds...")
                    time.sleep(backoff)
                    backoff *= 2  # Exponential backoff
                elif response.status_code == 503:
                    print("Server error. Retrying in 3 seconds...")
                    time.sleep(3)
                else:
                    print(response.text)
                    return None
            except SSLError as e:
                print(f"SSL Error encountered: {e}. Retrying in 10 seconds...")
                time.sleep(10)
        print(f"Failed to retrieve data after {retries} attempts.")
        counties_to_retry.append(params['locationid'])
        return None

    if num_days > 30:
        requests_list = []
        current_date = startdate
        while current_date <= enddate:
            current_enddate = current_date + pd.DateOffset(days=30)
            if current_enddate > enddate:
                current_enddate = enddate

            params = {
                "datasetid": "GHCND",
                "startdate": current_date.strftime("%Y-%m-%d"),
                "enddate": current_enddate.strftime("%Y-%m-%d"),
                "units": "standard",
                "limit": "1000",
                "locationid": f"FIPS:{state_ansi}{county_ansi}",
                'datatypeid': ['TMAX', 'TMIN']
            }

            print(f"FIPS:{state_ansi}{county_ansi}")
            result = make_request(params)
            if result is not None:
                requests_list.append(result)

            current_date = current_enddate + pd.DateOffset(days=1)

        if requests_list:
            return pd.concat(requests_list)
        else:
            return None
    else:
        params = {
            "datasetid": "GHCND",
            "startdate": startdate.strftime("%Y-%m-%d"),
            "enddate": enddate.strftime("%Y-%m-%d"),
            "units": "standard",
            "limit": "1000",
            "locationid": f"FIPS:{state_ansi}{county_ansi}",
            'datatypeid': ['TMAX', 'TMIN']
        }

        print(f"FIPS:{state_ansi}{county_ansi}")
        return make_request(params)

    # At the end of the process, write counties to a file if there are any
    if counties_to_retry:
        with open('counties_to_retry.txt', 'w') as file:
            for county in counties_to_retry:
                file.write(f"{county}\n")
        print("Counties with no data have been written to 'counties_to_retry.txt'.")

# Usage
# df = get_raw_weather(state_ansi, county_ansi, "2023-01-01", "2023-01-31")


In [72]:
years = range(2019, 2022)


for year in years:
  print(year)
  usda_data = get_usda_data(usda_key, year)
  usda_data.to_csv(f"usda_data_{year}.csv", index=False)
  weather_data=get_weather_data(usda_data)
  weather_data.to_csv(f"weather_data_{year}.csv", index=False)

2019
FIPS:26
FIPS:26
FIPS:26
FIPS:26
FIPS:26
FIPS:26
FIPS:26
FIPS:26
FIPS:26
FIPS:26
FIPS:26
FIPS:26
FIPS:26041
FIPS:26041
FIPS:26041
FIPS:26041
FIPS:26041
FIPS:26041
FIPS:26041
FIPS:26041
FIPS:26041
FIPS:26041
FIPS:26041
FIPS:26041
FIPS:26109
No data found for FIPS:FIPS:26109
FIPS:26109
No data found for FIPS:FIPS:26109
FIPS:26109
No data found for FIPS:FIPS:26109
FIPS:26109
No data found for FIPS:FIPS:26109
FIPS:26109
No data found for FIPS:FIPS:26109
FIPS:26109
No data found for FIPS:FIPS:26109
FIPS:26109
No data found for FIPS:FIPS:26109
FIPS:26109
No data found for FIPS:FIPS:26109
FIPS:26109
No data found for FIPS:FIPS:26109
FIPS:26109
No data found for FIPS:FIPS:26109
FIPS:26109
No data found for FIPS:FIPS:26109
FIPS:26109
No data found for FIPS:FIPS:26109
FIPS:26
FIPS:26
FIPS:26
Server error. Retrying in 3 seconds...
FIPS:26
FIPS:26
FIPS:26
FIPS:26
FIPS:26
FIPS:26
FIPS:26
FIPS:26
FIPS:26
FIPS:26019
FIPS:26019
FIPS:26019
FIPS:26019
FIPS:26019
FIPS:26019
FIPS:26019
FIPS:26019
FIPS